In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model

In [2]:
df = pd.read_csv('bbc_text_cls.csv')
df.head()

,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business


In [3]:
# map classes to integers from 0...K-1
df['labels'].astype("category").cat.codes

0       0
1       0
2       0
3       0
4       0
       ..
2220    4
2221    4
2222    4
2223    4
2224    4
Length: 2225, dtype: int8

In [4]:
df['targets'] = df['labels'].astype("category").cat.codes
df.head()

,text,labels,targets
0,Ad sales boost Time Warner profit\n\nQuarterly...,business,0
1,Dollar gains on Greenspan speech\n\nThe dollar...,business,0
2,Yukos unit buyer faces loan claim\n\nThe owner...,business,0
3,High fuel prices hit BA's profits\n\nBritish A...,business,0
4,Pernod takeover talk lifts Domecq\n\nShares in...,business,0


In [5]:
# df = df.drop(columns=['target'], axis=1)

In [6]:
# split the dataframe into train and test df
df_train, df_test = train_test_split(df, test_size=0.3)

In [7]:
tfidf = TfidfVectorizer(stop_words='english')
Xtrain = tfidf.fit_transform(df_train['text'])
Xtest = tfidf.fit_transform(df_test['text'])

In [8]:
Ytrain = df_train['targets']
Ytest = df_test['targets']

In [9]:
# Number of classes
K = df['targets'].max()+1
K

5

In [10]:
# input dimensions, number of columns
D = Xtrain.shape[1]
D

25247

In [11]:
# Build model
i = Input(shape=(D, ))
x = Dense(300, activation='relu')(i)
x = Dense(K)(x) # softmax include in loss

model = Model(i, x)

In [12]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 25247)]           0         
_________________________________________________________________
dense (Dense)                (None, 300)               7574400   
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 1505      
Total params: 7,575,905
Trainable params: 7,575,905
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'],
)

In [14]:
# Changing the data from sparse matrix to dense matrix
Xtrain = Xtrain.toarray()
Xtest = Xtest.toarray()

In [16]:
r = model.fit(
    Xtrain, 
    Ytrain, 
    validation_data=(Xtest, Ytest),
    epochs=7,
    batch_size=128,
             )

Epoch 1/7
13/13 [==============================] - ETA: 0s - loss: 0.9159 - accuracy: 0.9904

ValueError: in user code:

    C:\Users\hp\anaconda3\envs\env_ml\lib\site-packages\keras\engine\training.py:1330 test_function  *
        return step_function(self, iterator)
    C:\Users\hp\anaconda3\envs\env_ml\lib\site-packages\keras\engine\training.py:1320 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\hp\anaconda3\envs\env_ml\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\hp\anaconda3\envs\env_ml\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\hp\anaconda3\envs\env_ml\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\hp\anaconda3\envs\env_ml\lib\site-packages\keras\engine\training.py:1313 run_step  **